# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [2]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [3]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud' ; 
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [4]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

%matplotlib inline


In [5]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

For the model

In [6]:
model = 'eNATL60'
region = '1'
season = 'fma'
name_mod = model+'-Region'+region+'-'+season
name_ssh_mod = 'sossheig_bl'


For altimetry data

In [7]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [8]:
fresults=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod+'/result_??.nc'))
dsn=xr.open_mfdataset(fresults,concat_dim='time',combine='nested')
fresults2='../results/results_Altika-eNATL60-Region1-fma/result_gonzag_netcdf_nocloud.nc'
dsn2=xr.open_dataset(fresults2)

In [9]:
#clean up some remaining Nans

ds=dsn.where(nmp.isnan(dsn[name_ssh_sat])==0, drop=True)
ds2=dsn2.where(nmp.isnan(dsn2[name_ssh_sat])==0, drop=True)

In [10]:
ds2


<xarray.Dataset>
Dimensions:         (time: 10198)
Coordinates:
  * time            (time) datetime64[ns] 2014-02-02T23:14:13 ... 2014-04-29T...
Data variables:
    latitude        (time) float32 39.97 39.92 39.86 39.8 ... 30.15 30.1 30.04
    longitude       (time) float32 289.7 289.7 289.6 289.6 ... 287.5 287.5 287.5
    sossheig_np     (time) float32 0.5009 -0.273 -0.2974 ... 0.4733 0.4661 0.453
    sossheig_bl     (time) float32 0.5009 -0.2743 -0.2977 ... 0.4662 0.4528
    sla_unfiltered  (time) float32 -0.011 -0.001 0.026 ... 0.056 0.031 0.051
    distance        (time) float32 0.0 6.592 13.18 ... 1.094e+05 1.094e+05
Attributes:
    about:    Created with Gonzag package => https://github.com/brodeau/gonzag

In [11]:
print(ds[name_ssh_mod].values)

[-0.06541651  0.25607326  0.25922199 ... -0.21118741 -0.30652857
 -0.30651113]


In [12]:
ds2.time

<xarray.DataArray 'time' (time: 10198)>
array(['2014-02-02T23:14:13.000000000', '2014-02-02T23:14:13.980000000',
       '2014-02-02T23:14:14.960000000', ..., '2014-04-29T23:14:02.683653120',
       '2014-04-29T23:14:03.663652864', '2014-04-29T23:14:04.643652864'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2014-02-02T23:14:13 ... 2014-04-29T23:14:0...

### Time series of SSH 

In [14]:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = ds.time_counter ; 
VT2 = ds2.time ; 

fig = plt.figure(num = 1, figsize=(15,30), facecolor='w', edgecolor='k')
ax = plt.subplot(111)
plt.hlines(0,nmp.min(ds.time_counter.values),nmp.max(ds.time_counter.values), colors='k',label=None,  zorder=5)
plt.plot(VT, ds[name_ssh_sat]-nmp.mean(ds[name_ssh_sat]), '.', color=clr_sat, markersize=5, 
         alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, ds[name_ssh_mod]-nmp.mean(ds[name_ssh_mod]), '.', color=clr_mod, markersize=5, 
         alpha=0.5, label=name_mod, zorder=15)
plt.plot(VT2, ds2[name_ssh_mod]-nmp.mean(ds2[name_ssh_mod]), '.', color='g', markersize=5, 
         alpha=0.5, label=name_mod+'-netcdf', zorder=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation='60')
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)

plt.savefig('../plots/SSH_'+name_sat+'-'+name_mod+'_cloud-netcdf.png')
del fig,ax

In [15]:
clr_sat = '#AD0000'
clr_mod = '#008ab8'

VT = ds.time_counter ; 
VT2 = ds2.time ; 

fig = plt.figure(num = 1, figsize=(15,30), facecolor='w', edgecolor='k')
ax = plt.subplot(111)
plt.hlines(0,nmp.min(ds.time_counter.values),nmp.max(ds.time_counter.values), colors='k',label=None,  zorder=5)
plt.plot(VT, ds[name_ssh_sat], '.', color=clr_sat, markersize=5, 
         alpha=0.5, label=name_sat, zorder=10)
plt.plot(VT, ds[name_ssh_mod], '.', color=clr_mod, markersize=5, 
         alpha=0.5, label=name_mod, zorder=15)
plt.plot(VT2, ds2[name_ssh_mod], '.', color='g', markersize=5, 
         alpha=0.5, label=name_mod+'-netcdf', zorder=15)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xticks(rotation='60')
plt.ylabel('SSH [m]')
ax.grid(color='k', linestyle='-', linewidth=0.3)
lgnd = plt.legend(bbox_to_anchor=(0.55, 1.), ncol=1, shadow=True, fancybox=True)

plt.savefig('../plots/SSH_nomean_'+name_sat+'-'+name_mod+'_cloud-netcdf.png')
del fig,ax